In [ ]:
# Siti 224 swin v2 (augments: Flip(), Rotate(max_deg=10))
# Baseline is 17.55 cv
#beta 0.2 17.49
#beta 0.3 17.450
#beta 0.4 17.4825
#beta 0.5 17.4835
#beta 0.6 17.496
#beta 0.7 17.501

In [ ]:
SEED = 365

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
from timm import create_model
from timm.data.mixup import Mixup

from fastai.vision.all import *

In [ ]:
set_seed(SEED, reproducible=True)
BATCH_SIZE = 8
NEED_TRAIN = True

In [ ]:
train_df = pd.read_csv('../input/no-dupes-pawpularity-rice-folds/train_5ricefolds.csv')
dataset_path = Path('../input/petfinder-pawpularity-score/train/')
train_df['path'] = train_df['Id'].map(lambda x: str(dataset_path/x) + '.jpg')
train_df = train_df.drop(columns=['Id'])
train_df = train_df.sample(frac=1).reset_index(drop=True) #shuffle dataframe

In [ ]:
train_df['norm_score'] = train_df['Pawpularity']/100

In [ ]:
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
    os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'

In [ ]:
seed=SEED
set_seed(seed, reproducible=True)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

In [ ]:
def petfinder_rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

In [ ]:
"""# Phase 1 - large step values

import gc

from sklearn.metrics import mean_squared_error

folds = [0,1,2,3,4]
beta_vals = [0.2,0.3,0.4,0.5,0.6,0.7]
avg_rmse_list = []

for beta_val in beta_vals:
    print(f'Beta: {beta_val}')
    if NEED_TRAIN:
        rmse_list = []
        #train_df['oof'] = -1

        for i in folds:

            print(f'Fold {i} results')

            # ----------------
            # Change Path here
            # ----------------
            learn  = load_learner(fname=Path(f'../input/fastai-mixup-2/model_fold_{i}.pkl'), cpu=False)

            val_idx = train_df[train_df['kfold'] == i].index
            val_df = train_df.loc[val_idx]

            test_dl = learn.dls.test_dl(val_df)

            val_pred, _ =learn.tta(dl=test_dl,n=5, beta=beta_val)
            val_pred = val_pred.numpy()
            val_pred = val_pred * 100

            train_df.loc[val_idx, 'oof'] = val_pred
            #print('Pawpularity vs OOF')
            #print(val_df['Pawpularity'][:5], val_pred[:5])

            real_rmse = mean_squared_error(train_df[train_df['kfold'] == i]['Pawpularity'], 
                        train_df[train_df['kfold'] == i]['oof'], squared=False)

            fold_rmse = mean_squared_error(train_df[train_df['kfold'] == i]['oof'], train_df[train_df['kfold'] == i]['Pawpularity'], squared=False)
            print(f'Fold {i} RMSE: {fold_rmse}')
            rmse_list.append(fold_rmse)

            del learn

            torch.cuda.empty_cache()

            gc.collect()
    
    train_df.to_csv(f'siti_oof_v2_beta_{beta_val}.csv')
    
    tta_rmse = sum(rmse_list) / len(rmse_list)
    avg_rmse_list.append(tta_rmse)
    print(f'Average Fold RMSE for beta {beta_val}: {tta_rmse}')
    
print(f'Beta vals: {beta_vals}')
print(f'RMSE vals: {avg_rmse_list}')"""

In [ ]:
# Optuna script to find best float beta value

import optuna

import gc

from sklearn.metrics import mean_squared_error

folds = [0,1,2,3,4]

def objective(trial):
    if NEED_TRAIN:
        rmse_list = []
        #train_df['oof'] = -1

        for i in folds:

            #print(f'Fold {i} results')
            
            # ----------------
            # Change Path here
            # ----------------
            learn  = load_learner(fname=Path(f'../input/fastai-mixup-v4/model_fold_{i}.pkl'), cpu=False)
            
            val_idx = train_df[train_df['kfold'] == i].index
            val_df = train_df.loc[val_idx]

            test_dl = learn.dls.test_dl(val_df)

            # hyperopt here
            beta = trial.suggest_loguniform('beta', 0.35, 0.55)
            
            val_pred, _ =learn.tta(dl=test_dl,n=5, beta=beta)
            val_pred = val_pred.numpy()
            val_pred = val_pred * 100

            train_df.loc[val_idx, 'oof'] = val_pred
            #print('Pawpularity vs OOF')
            #print(val_df['Pawpularity'][:5], val_pred[:5])

            real_rmse = mean_squared_error(train_df[train_df['kfold'] == i]['Pawpularity'], 
                        train_df[train_df['kfold'] == i]['oof'], squared=False)

            fold_rmse = mean_squared_error(train_df[train_df['kfold'] == i]['oof'], train_df[train_df['kfold'] == i]['Pawpularity'], squared=False)
            #print(f'Fold {i} RMSE: {fold_rmse}')
            rmse_list.append(fold_rmse)

            del learn

            torch.cuda.empty_cache()

            gc.collect()

    train_df.to_csv(f'siti_oof_v2_beta_{beta}.csv')
    tta_rmse = sum(rmse_list) / len(rmse_list)
    #print(f'Average Fold RMSE: {fold_rmse}')
    return tta_rmse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=15)

In [ ]:
print('Best beta val')
study.best_params

In [ ]:
#train_df[['oof','Pawpularity']].plot.scatter('oof','Pawpularity')

In [ ]:
#train_df[['oof','Pawpularity']].hist()

In [ ]:
#train_df.to_csv('train_with_oof.csv')